In [ ]:
import os
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
# Verify whether image directory exists and the form is the correct form.

per_image = 468
dataset_path = "../../data"
assert os.path.exists(os.path.join(dataset_path, "patchPoseA"))
assert os.path.exists(os.path.join(dataset_path, "patchPoseB"))
print(os.listdir(os.path.join(dataset_path, "patchPoseA"))[0])
print(os.listdir(os.path.join(dataset_path, "patchPoseB"))[0])

In [ ]:
# PatchPoseA, PatchPoseB, data number verification
# The number of whole images in the data directory must be divisible to the patches extracted per image. 


print(len(os.listdir(os.path.join(dataset_path, "patchPoseA"))))
print(len(os.listdir(os.path.join(dataset_path, "patchPoseB"))))

assert len(os.listdir(os.path.join(dataset_path, "patchPoseA"))) % per_image == 0
assert len(os.listdir(os.path.join(dataset_path, "patchPoseB"))) % per_image == 0


In [ ]:
# You could change the idx to observe images
idx_to_observe = 0

lines = open(os.path.join(dataset_path, "patchPoseA.txt")).readlines()
fig, axs = plt.subplots(6, 6, figsize=(20, 20))
for i in range(per_image // 13):
    image = Image.open(os.path.join(dataset_path, lines[13 * i + 468 * idx_to_observe].split(' ')[1]))
    axs[i // 6, i % 6].imshow(image)
plt.show()

fig, axs = plt.subplots(1, 13, figsize=(20, 20))
for i in range(per_image // 36):
    image = Image.open(os.path.join(dataset_path, lines[i + 468 * idx_to_observe].split(' ')[1]))
    axs[i].imshow(image)
plt.show()
    

In [ ]:
# Check train: val: test
patchPoseA_list_dir = os.path.join(dataset_path, "patchPoseAImageList")
patchPoseB_list_dir = os.path.join(dataset_path, "patchPoseAImageList")

assert os.path.exists(patchPoseA_list_dir)
assert os.path.exists(patchPoseB_list_dir)

trainAfp, valAfp, testAfp = (
    open(os.path.join(patchPoseA_list_dir, "train_acquired.txt")), 
    open(os.path.join(patchPoseA_list_dir, "val_acquired.txt")), 
    open(os.path.join(patchPoseA_list_dir, "test_acquired.txt")),
)

trainA, valA, testA = (
    trainAfp.readlines(), 
    valAfp.readlines(),
    testAfp.readlines(),
)

trainBfp, valBfp, testBfp = (
    open(os.path.join(patchPoseB_list_dir, "train_acquired.txt")),
    open(os.path.join(patchPoseB_list_dir, "val_acquired.txt")), 
    open(os.path.join(patchPoseB_list_dir, "test_acquired.txt")),
)

trainB, valB, testB = (
    trainBfp.readlines(), 
    valBfp.readlines(),
    testBfp.readlines(),
)

allA = len(trainA) + len(valA) + len(testA)
allB = len(trainB) + len(valB) + len(testB)

print(f"train: val: test = {len(trainA) / allA} : {len(valA) / allA} : {len(testA) / allA}")
print(f"train: val: test = {len(trainB) / allB} : {len(valB) / allB} : {len(testB) / allB}")

trainAfp.close(); valAfp.close(); testAfp.close(); 
trainBfp.close(); valBfp.close(); testBfp.close(); 


In [ ]:
# You could change the idx to observe images
idx_to_observe = 2

lines = open(os.path.join(patchPoseA_list_dir, "train_acquired.txt")).readlines()
print(len(lines))
fig, axs = plt.subplots(5, 10, figsize=(120, 60))
for i in range(50):
    image = Image.open(os.path.join(dataset_path, lines[468 * i].rstrip("\n")))
    axs[i // 10, i % 10].imshow(image)
    

In [ ]:
# You could change the idx to observe images

lines = open(os.path.join(patchPoseA_list_dir, "train_pruned.txt")).readlines()
fig, axs = plt.subplots(5, 10, figsize=(120, 60))
for i in range(50):
    image = Image.open(os.path.join(dataset_path, lines[468 * i].rstrip("\n")))
    axs[i // 10, i % 10].imshow(image)
plt.show()

    

In [ ]:
# Evaluating Performance
import numpy as np
import tqdm

path = "/user4/yoonwoo/output_sift/output_feat.txt.result.npz"
npz_file = np.load(path, allow_pickle=True)
thresholds = 5

In [ ]:
angle_separate, sca_separate, both_separate = {}, {}, {}
for angle in range(0, 360, 10):
    angle_separate[angle] = []
for scale in [round(2 ** ((i - 6) / 3), 4) for i in range(13)]:
    sca_separate[scale] = []
for angle in range(0, 360, 10): 
    for scale in [round(2 ** ((i - 6) / 3), 4) for i in range(13)]:
        both_separate[(angle, scale)] = []
        
for key,val in tqdm.tqdm(npz_file.items()):
    ori = int(key.split("/")[-1].split("_")[0])
    sca = float(key.split("/")[-1].split("_")[1][:-len(".ppm")])
    score = (val < thresholds).astype(float).mean()
    sca_separate[sca].append(score)
    angle_separate[ori].append(score)
    both_separate[(ori, sca)].append(score)

In [ ]:
angle_mean = {k: np.mean(v) for (k,v) in angle_separate.items()}
sca_mean = {k: np.mean(v) for (k,v) in sca_separate.items()}
both_mean = {k: np.mean(v) for (k,v) in both_separate.items()}

In [ ]:
angle_list = np.array(list(angle_mean.values())).reshape(1, -1)
sca_list = np.array(list(sca_mean.values())).reshape(1, -1)
both_list = np.array(list(both_mean.values())).reshape(36, 13)

In [ ]:
import matplotlib.pyplot as plt
angle_list

In [ ]:
# Scale Accuracy Color Map
fig, ax = plt.subplots(figsize=(40,6))
im = ax.imshow(angle_list, cmap="Blues")
ax.set_xticks(np.arange(36))
ax.set_yticks(np.arange(1))
ax.set_xticklabels(list(map(str, range(0, 360, 10))))
for i in range(36):
    ax.text(i, 0, round(angle_list[0][i], 4), ha="center", va="center", color="b")
plt.show()

In [ ]:
# Scale Accuracy Color Map
fig, ax = plt.subplots(figsize=(40,6))
im = ax.imshow(sca_list, cmap="Blues")
ax.set_xticks(np.arange(13))
ax.set_yticks(np.arange(1))
ax.set_xticklabels(list(map(str, [round(2 ** ((i - 6) / 3), 4) for i in range(13)])))
for i in range(13):
    ax.text(i, 0, round(sca_list[0][i], 4), ha="center", va="center", color="b")
plt.show()

In [ ]:
# Scale Accuracy Color Map
fig, ax = plt.subplots(figsize=(40,50))
im = ax.imshow(both_list, cmap="Blues")
ax.set_yticks(np.arange(36))
ax.set_xticks(np.arange(13))
ax.set_yticklabels(range(0, 360, 10))
ax.set_xticklabels(list(map(str, [round(2 ** ((i - 6) / 3), 4) for i in range(13)])))
for i in range(36):
    for j in range(13):
        ax.text(j, i, round(both_list[i][j], 4), ha="center", va="center", color="b")
plt.show()